## Now that dependencies are installed, ready to move on!

## Log into huggingface
Run the cell below and paste your token into the prompt.  You can get your token from your [huggingface account page](https://huggingface.co/settings/tokens).

The token will not show on the screen, just press enter after you paste it.

Then run the following cell to download the base checkpoint (may take a minute).

In [ ]:
from huggingface_hub import notebook_login, hf_hub_download
import os
notebook_login()

In [ ]:
repo="panopstor/EveryDream"
ckpt_file="sd_v1-5_vae.ckpt"

print(f"Downloading {ckpt_file} from {repo}")
downloaded_model_path = hf_hub_download(repo, ckpt_file, cache_dir="/workspace/hfcache")
!cp -L {downloaded_model_path} /workspace/EveryDream2trainer

ckpt_name = os.path.splitext(ckpt_file)[0]

if not os.path.exists(f"ckpt_cache/{ckpt_name}"):
    !python utils/convert_original_stable_diffusion_to_diffusers.py --scheduler_type ddim \
    --original_config_file v1-inference.yaml \
    --image_size 512 \
    --checkpoint_path "{ckpt_file}" \
    --prediction_type epsilon \
    --upcast_attn False \
    --dump_path "ckpt_cache/{ckpt_name}"

print("DONE")

# Start Training
Naming your project will help you track what the heck you're doing when you're floating in checkpoint files later.

You may wish to consider adding "sd1" or "sd2v" or similar to remember what the base was, as you'll also have to tell your inference app what you were using, as its difficult for programs to know what inference YAML to use automatically. For instance, Automatic1111 webui requires you to copy the v2 inference YAML and rename it to match your checkpoint name so it knows how to load the file, tough it assumes SD 1.x compatible. Something to keep in mind if you start training on SD2.1.

Next cell runs training. This will take a while depending on your number of images, repeats, and max_epochs.

You can watch for test images in the logs folder.

In [ ]:
project_name = "finetune_v1"

!python train.py --project_name "{project_name}_384_35e6" \
--resume_ckpt "{ckpt_name}" \
--data_root "input" \
--resolution 384 \
--batch_size 5 \
--max_epochs 100 \
--save_every_n_epochs 9999 \
--lr 3.5e-6 \
--lr_scheduler cosine \
--sample_steps 100 \
--ed1_mode \
--useadam8bit \
--save_full_precision \
--shuffle_tags \
--notebook

!python train.py --project_name "{project_name}_512_25e6" \
--resume_ckpt "findlast" \
--data_root "input" \
--max_epochs 50 \
--ckpt_every_n_minutes 60 \
--resolution 512 \
--batch_size 4 \
--gradient_checkpointing \
--lr 2.5e-6 \
--lr_scheduler cosine \
--sample_steps 200 \
--ed1_mode \
--useadam8bit \
--save_full_precision \
--shuffle_tags \
--notebook

print("DONE")

In [ ]:
!python train.py --project_name "{project_name}_512_25e6" \
--resume_ckpt "findlast" \
--data_root "input" \
--save_every_n_epochs  \
--resolution 512 \
--batch_size 4 \
--gradient_checkpointing \
--max_epochs 10 \
--lr 2.5e-6 \
--lr_scheduler cosine \
--sample_steps 200 \
--ed1_mode \
--useadam8bit \
--save_full_precision \
--shuffle_tags \
--notebook

# HuggingFace upload
Use the cell below to upload your checkpoint to your personal HuggingFace account if you want instead of manually downloading. You should already be authorized to Huggingface by token if you used the download/token cells above.

Make sure to fill in the three fields at the top. This will only upload one file at a time, so you will need to run the cell below for each file you want to upload.

* You can get your account name from your [HuggingFace account page](https://huggingface.co/settings/account). Look for your "username" field and paste it below.

* You only need to setup a repository one time.  You can create it here: [Create New HF Model](https://huggingface.co/new)  Make sure you write down the repo name you make for future use.  You can reuse it later.

* You need to type the name of the ckpts one at a time in the cell below, find them in the left file drawer of your Runpod/Vast/Colab.

In [ ]:
#list ckpts in root that are ready for download
import glob

for f in glob.glob("*.ckpt"):
    print(f)

In [ ]:
!huggingface-cli lfs-enable-largefiles
# fill in these three fields:
hfusername = "MyHfUser"
reponame = "MyRepo"
ckpt_name = "finetune_v1-ep10-gs02500.ckpt"


target_name = ckpt_name.replace('-','').replace('=','')
import os
os.rename(ckpt_name,target_name)
repo_id=f"{hfusername}/{reponame}"
print(f"uploading to HF: {repo_id}/{ckpt_name}")
print("this make take a while...")

from huggingface_hub import HfApi
api = HfApi()
response = api.upload_file(
    path_or_fileobj=target_name,
    path_in_repo=target_name,
    repo_id=repo_id,
    repo_type=None,
    create_pr=1,
)
print(response)
print(finish_msg)
print("go to your repo and accept the PR this created to see your file")